In [ ]:
import pandas as pd
import pymongo
import pprint
from collections import Counter
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [ ]:
from pymongo import MongoClient
client = MongoClient()

In [ ]:
#db = client.hkust
db = client.msbd500103

### Load Training Data

In [ ]:
trData = db.training_record
#trData = db.training_record

In [ ]:
trData = trData.find({})
trData = list(trData)
df_trData = pd.DataFrame(trData)
df_trData.head()

In [ ]:
df_trData = df_trData[~df_trData['user_gp_frequency'].isnull()]
df_trData = df_trData[~df_trData['article_title'].isnull()]

In [ ]:
#sophia remark get the mapping of user article
df_ua_map = df_trData[['user_id', 'article_contentid']]
df_ua_map.shape

In [ ]:
df_ua_map = df_ua_map.drop_duplicates(subset = ['user_id', 'article_contentid'], keep='first')
df_ua_map.shape

### Load Test User

In [ ]:
tstData = db.test_user
tstData = tstData.find({})
tstData = list(tstData)
df_tstData = pd.DataFrame(tstData)
df_tstData.head()

In [ ]:
#df_tstData.drop_duplicates(['user_id'])['user_id'].to_csv('data1202/df_tst_user.csv', sep='\t', encoding='utf-8')
#df_tstData.drop_duplicates(['user_id'])[['user_id', 'user_age','user_gender']].to_csv('data1202/df_tst_user_d.csv', sep='\t', encoding='utf-8')

In [ ]:
df_tstData.drop_duplicates(['user_id']).shape

### Prepare User View: df_tr_user vs df_tst_user¶

In [ ]:
df_tr_user = df_trData[['user_id', 'user_age', 'user_gender','user_maxmind_city']]
df_tst_user = df_tstData[['user_id', 'user_age', 'user_gender', 'user_maxmind_city']]
df_tr_user.head()

In [ ]:
df_tst_user.head()

In [ ]:
df_user = pd.concat([df_tr_user, df_tst_user])

In [ ]:
df_user['user_age'] = df_user['user_age'].fillna(0)
df_user['user_gender'] = df_user['user_gender'].fillna(0)
#df_user['user_brand'] = df_user['user_brand'].str.lower().fillna('na')
df_user['user_maxmind_city'] = df_user['user_maxmind_city'].str.lower().fillna('na')
#df_user['user_model'] = df_user['user_model'].str.lower().fillna('na')
df_user.head()

In [ ]:
df_user = pd.get_dummies(df_user, columns=['user_age', 'user_gender', 'user_maxmind_city'])
df_user = df_user.fillna(0)
df_user = df_user.drop_duplicates(subset=['user_id'], keep='first')
df_user.head()

In [ ]:
train_user_id = df_trData.drop_duplicates(subset=['user_id'], keep='first')['user_id'].values
test_user_id = df_tstData.drop_duplicates(subset=['user_id'], keep='first')['user_id'].values

In [ ]:
df_user.shape

In [ ]:
df_tr_user = df_user[df_user['user_id'].isin(train_user_id)]
df_tr_user.shape

In [ ]:
df_tst_user = df_user[df_user['user_id'].isin(test_user_id)]
df_tst_user.shape

### Prepare App View

In [ ]:
df_tr_app = df_trData[['user_id', 'user_gp_frequency']]
df_tr_app.head()

In [ ]:
df_tst_app = df_tstData[['user_id', 'user_gp_frequency']]

In [ ]:
df_tst_na_app = df_tst_app[df_tst_app['user_gp_frequency'].isnull()]
df_tst_na_app = df_tst_na_app.drop(['user_gp_frequency'],axis = 1)
df_tst_na_app['NOAPP'] = 1
df_tst_na_app.shape

In [ ]:
df_app = pd.concat([df_tr_app, df_tst_app])
df_app = df_app.dropna()

In [ ]:
df_app_category = pd.DataFrame.from_records(df_app['user_gp_frequency'].tolist())
df_app_category = df_app_category.fillna(0)
df_app_category['NOAPP'] = 0
df_app_category.head()

In [ ]:
df_app = df_app.merge(df_app_category, how='left', left_index=True, right_index=True)
df_app = df_app.drop(['user_gp_frequency'], axis=1)
df_app = df_app.drop_duplicates(subset=['user_id'], keep='first')
df_app.head()

In [ ]:
df_app = pd.concat([df_app, df_tst_na_app])
df_app.head()


In [ ]:
def sumCat(row):
    l = row.tolist()
    #print l
    row['totalCat'] = sum(l[1:])
    return row

In [ ]:
df_app1 = df_app.drop(['user_id'], axis = 1).apply(lambda row: sumCat(row), axis = 1)
df_app1 = df_app[['user_id']].merge(df_app1, how = 'left', left_index=True, right_index=True)
df_app2 = df_app1[df_app1['totalCat'] != 0].drop(['totalCat'], axis = 1)

In [ ]:
df_app2.shape

In [ ]:
df_app2 = df_app2.drop_duplicates(subset=['user_id'], keep='first')

In [ ]:
df_app2.shape

In [ ]:
df_app2.head()

In [ ]:
app_category = df_app.drop(['user_id'], axis = 1).columns
app_category

## Merge user and app

In [ ]:
df_tr_user.shape, df_app2.shape

In [ ]:
df_tr_user_app = df_tr_user.merge(df_app2, how = 'left', on = 'user_id')
df_tr_user_app = df_tr_user_app.fillna(0) #change from fillna(0) to fillna(0.1)
df_tr_user_app.head()

In [ ]:
df_tst_user_app = df_tst_user.merge(df_app2, how='left',on='user_id')
df_tr_user_app = df_tr_user_app.fillna(0)
df_tr_user_app.head()

### Load Test Article

In [ ]:
tstArt = db.test_article
tstArt = tstArt.find({})
tstArt = list(tstArt)
df_tstArt = pd.DataFrame(tstArt)
df_tstArt.head()

In [ ]:
#df_tstArt.drop_duplicates(['article_contentid'])['article_contentid'].to_csv('data1202/df_tst_art.csv', sep = '\t', encoding='utf-8')
df_tstArt.drop_duplicates(['article_contentid'])[['article_contentid','article_publisher','article_update_time']].to_csv('data1202/df_tst_art_1.csv', sep = '\t', encoding='utf-8')

### Prepare Article View

In [ ]:
df_tr_article = df_trData[['user_id', 'article_contentid', 'article_l1_categories', 'article_l2_categories', 'article_title', 'dt']]#sophia remark : should add article_update_time as a feature
df_tr_article = df_tr_article.sort_values(by=['article_title'], ascending=False)
df_tr_article.head()

In [ ]:
from sklearn import preprocessing
#Create x, where x the 'scores' column's values as floats
y = df_tr_article['dt'].values.astype(float)
y = y.reshape(-1,1)
# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler()

# Create an object to transform the data to fit minmax processor
y_scaled = min_max_scaler.fit_transform(y)
y_scaled
# Run the normalizer on the dataframe
df_tr_article['dt'] = y_scaled

In [ ]:
df_tst_article = df_tstArt[['article_contentid', 'article_l1_categories', 'article_l2_categories', 'article_title']]#sophia remark : should add article_update_time as a feature
df_tst_article = df_tst_article.sort_values(by=['article_title'], ascending=False)
df_tst_article = df_tst_article.drop_duplicates(['article_contentid'])
df_tst_article.head()

In [ ]:
df_tst_article.shape

In [ ]:
df_article_temp = df_tr_article[['article_contentid', 'dt']].drop_duplicates(['article_contentid'])

df_tst_article = df_tst_article.merge(df_article_temp[['article_contentid', 'dt']], how = 'left', on = 'article_contentid')
df_tst_article.head()

In [ ]:
df_tst_article['dt'] = df_tst_article['dt'].fillna(1)
df_tst_article.head()

In [ ]:
df_tst_article.shape

In [ ]:
df_tst_article['user_id'] = 'na'

In [ ]:
#df_tst_article.head()

In [ ]:
tst_article_list = df_tst_article['article_contentid'].values

In [ ]:
df_tr_article = df_tr_article.drop_duplicates(subset=['article_contentid'], keep='first')
df_tr_article = df_tr_article.reset_index(drop = True)
df_tr_article.head()

In [ ]:
df_tr_article.shape

In [ ]:
df_article = pd.concat([df_tr_article, df_tst_article])
df_article.head()

In [ ]:
tr_article_list = df_tr_article['article_contentid'].values

In [ ]:
def getDetail(cat, field):
    if (str(cat) != 'nan'):
        if (len(cat) > 0):
            return cat[0][field]
    else:
        return '0'

In [ ]:
df_article['l1_cat'] = df_article.apply(lambda row: getDetail(row['article_l1_categories'],'name'), axis=1)
df_article['l2_cat'] = df_article.apply(lambda row: getDetail(row['article_l2_categories'],'name'), axis=1)
df_article['l1_cat_w'] = df_article.apply(lambda row: getDetail(row['article_l1_categories'],'weight'), axis=1)
df_article['l2_cat_w'] = df_article.apply(lambda row: getDetail(row['article_l2_categories'],'weight'), axis=1)
df_article = df_article.drop(['article_l1_categories', 'article_l2_categories'], axis=1)
df_article.head()

In [ ]:
df_article.shape

In [ ]:
df_article.drop(['article_title'], axis = 1).to_csv('data1202/df_article.csv')

### Clean title

In [ ]:
#skip

from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag
from nltk.stem.lancaster import LancasterStemmer
import re


In [ ]:
def hashingTitle(title):
    STEMMER = LancasterStemmer()
    if (type(title) != float) & (type(title) != int):
        title = title.lower()
        title = word_tokenize(title)
        title = " ".join([STEMMER.stem(word.strip('!"#$%&\()*+,-./:;<=>?@[\]^_`{|}~').lower()) for word in title])
        title = pos_tag(title.split())
        title = ([w[0] for w in title if w[1] in ['NN', 'NNP']])
        titleList = []
        for w in title:
            if len(re.findall("[a-zA-Z]+", w)) > 0:
                f_w = re.findall("[a-zA-Z]+", w)[0].lower()
                f_w = '#'+f_w+'#'
                f_w_tri = [f_w[i:i+3] for i in range(len(f_w)-1) if (len(f_w[i:i+3]) == 3)]
                titleList.extend(f_w_tri)
        return dict(Counter(titleList))
    else:
        return {}

In [ ]:
df_article['clean_article_title'] = df_article.apply(lambda x: hashingTitle(x['article_title']), axis=1)
df_article.head()

In [ ]:
df_article_title = pd.DataFrame.from_records(df_article['clean_article_title'].tolist())
df_article_title = df_article_title.fillna(0)
df_article_title.head()

In [ ]:
df_article = df_article.merge(df_article_title, how='left', left_index=True, right_index=True)
df_article.head()

In [ ]:
#df_article = df_article.drop(['article_title','clean_article_title'], axis=1)
#Sophia commend : the following line is to be run if title is not included
df_article = df_article.drop(['article_title'], axis=1)
df_article = df_article.dropna()
df_article.head()

In [ ]:
df_cat = df_article[['l1_cat', 'l2_cat', 'l1_cat_w', 'l2_cat_w']]
df_cat = pd.get_dummies(df_cat, columns=['l1_cat', 'l2_cat'])
df_cat.head()

In [ ]:
df_cat = df_cat.merge(df_article[['l1_cat', 'l2_cat']], how='left', left_index=True, right_index=True)
df_cat.head()

In [ ]:
def updateCatWeight(row):
    cat = row['l1_cat']
    row['l1_cat_'+str(cat)] = row['l1_cat_w']
    return row

In [ ]:
df_cat = df_cat.apply(lambda row: updateCatWeight(row), axis=1)
df_cat.head()

In [ ]:
df_cat = df_cat.drop(['l1_cat','l2_cat','l1_cat_w', 'l2_cat_w'], axis=1)
df_cat.head()

In [ ]:
df_article = df_article.merge(df_cat, how='left', left_index=True, right_index=True)
#sophia remark: commend here
#df_article = df_article.drop(['article_publisher','article_title'], axis = 1)
df_article.head()

In [ ]:
df_article_2 = df_article.drop(['l1_cat','l2_cat','l1_cat_w','l2_cat_w'], axis = 1)
df_article_2.shape

In [ ]:
df_tr_article_2=df_article_2[df_article_2['article_contentid'].isin(tr_article_list)].drop_duplicates(['article_contentid'])
df_tst_article_2=df_article_2[df_article_2['article_contentid'].isin(tst_article_list)].drop_duplicates(['article_contentid'])

In [ ]:
print df_tr_article_2.shape
df_tr_article_2 = df_tr_article_2[df_tr_article_2['user_id'] != 'na']
df_tr_article_2.head()

In [ ]:
print df_tst_article_2.shape
df_tst_article_2 = df_tst_article_2.drop(['user_id'],axis = 1)
df_tst_article_2.head()

### Prepare Train Test Set

In [ ]:
df_train_user_0 = df_tr_user_app
df_test_user_0 = df_tst_user_app.fillna(0)
df_tr_user_app.shape, df_tst_user_app.shape

In [ ]:
user_fea_col = [i for i in df_train_user_0.columns \
                if i not in app_category and 'user_brand' not in i\
                and 'user_model' not in i  and 'article_contentid' != i]
len(user_fea_col)

In [ ]:
df_test_user = df_test_user_0[user_fea_col]
df_test_user.head()

In [ ]:
df_train_user = df_train_user_0[user_fea_col]
df_train_user.head()

In [ ]:
df_test_user.shape, df_train_user.shape

In [ ]:
#df_train_user = df_user[df_user['user_id'].isin(final_train_user_id)]
#df_train_user = df_train_user.drop(['user_id'], axis=1)
#df_train_user_2 = df_train_user[user_fea_col].fillna(0)
#df_train_user_arr = df_train_user_2.values
#df_train_user_arr.shape

In [ ]:
#Sophia remark: get test user list
#df_test_user0 = df_user[df_user['user_id'].isin(final_test_user_id)]

In [ ]:
user_fea_col

In [ ]:
#df_test_user = df_user[df_user['user_id'].isin(final_test_user_id)]
#df_test_user = df_test_user.drop(['user_id'], axis=1)
df_test_user_2 = df_test_user[user_fea_col].fillna(0)
#df_test_user_2 = df_test_user.fillna(0)
df_test_user_arr = df_test_user_2.values
df_test_user_arr.shape

In [ ]:
app_category

In [ ]:
df_test_app0 = df_app2[df_app2['user_id'].isin(test_user_id)]
df_test_app_2 = df_test_user_0[['user_id']].merge(df_test_app0, how = 'left', on = 'user_id')
df_test_app_2.head()

In [ ]:
#Question: test user without app category
df_test_app_2 = df_test_app_2.fillna(0.01)
df_test_app_arr = df_test_app_2.values
df_test_app_arr.shape

In [ ]:
df_test_user_2 = df_test_user_2.merge(df_test_app_2, how = 'left', on = 'user_id')
#df_test_user_2 = df_test_user_2.drop(['user_id'],axis = 1)
df_test_user_arr = df_test_user_2.values
df_test_user_arr.shape

In [ ]:
# may need to do sampling on df_ua_map
###sophia 20171201
#df_ua_map = df_ua_map.sample(frac=0.3, replace=False)

In [ ]:
len(df_ua_map)

In [ ]:
#df_train_user = df_train_user_0.merge(df_user_cat, how = 'left', on = 'user_id')

In [ ]:
df_final_train_user_article = df_ua_map[(df_ua_map['user_id'].isin(train_user_id))]
df_final_train_user_article = df_final_train_user_article.merge(df_train_user[user_fea_col], how = 'left', on = 'user_id')

In [ ]:
print (df_final_train_user_article.shape)
df_final_train_user_article.head()

In [ ]:
df_final_train_user_article = df_final_train_user_article.merge(df_tr_article_2.drop(['user_id'],axis = 1), how = 'left', on = 'article_contentid')
df_final_train_user_article.head()

In [ ]:
df_final_train_user_article.shape

In [ ]:
#df_final_train_user_article.dropna().shape

In [ ]:
#df_final_train = df_final_train_user_article.merge(df_user, how='left', on='user_id')
df_final_train = df_final_train_user_article.dropna()
#sophia remark: add app data
#df_final_train = df_final_train.merge(df_app2, how='left', on='user_id')

In [ ]:
df_final_train.shape

In [ ]:
df_final_train = df_final_train.merge(df_app2, how='left', on='user_id')
###sophia remark, check wether there is NAN or all 0 here
#df_final_train = df_final_train.dropna()
#df_final_train.head()

In [ ]:
df_final_train.shape

In [ ]:
article_feature = [i for i in df_final_train.columns if 'user_' not in i and i not in ['article_contentid',
 'article_title',
 'article_publisher',
 'l1_cat',
 'l2_cat',
 'l1_cat_w',
 'l2_cat_w',
 'l1_cat_w_x',
 'l2_cat_w_x',
 'l1_cat_w_y',
 'l2_cat_w_y',
 'totalCat'] and i not in app_category]
#article_feature = 
article_feature

In [ ]:
uapp_feature = [i for i in df_test_user_2.columns if i not in article_feature]
len(uapp_feature)

In [ ]:
df_train_user = df_final_train[uapp_feature]
df_train_user.head()

In [ ]:
df_train_app = df_final_train[app_category]
df_train_app.head()

In [ ]:
df_train_article = df_final_train[article_feature]
df_train_article.head()

In [ ]:
df_test_article = df_tst_article_2[article_feature]
#df_test_article.shape

In [ ]:
article_feature.append(u'article_contentid')
#len(article_feature)

In [ ]:
df_test_article0 = df_tst_article_2.drop_duplicates(subset=['article_contentid'], keep='first')#[article_feature]
df_test_article0.head()

In [ ]:
df_test_article_arr1 = df_test_article.values

In [ ]:
y_train = [1 for i in range(len(df_train_user.values))]
len(y_train)

### Build Model By Keras (User Vs App Vs Article)

In [ ]:
df_train_user_arr = df_train_user.values
df_train_app_arr = df_train_app.values

In [ ]:
df_train_user_arr.shape, df_train_app_arr.shape

In [ ]:
df_train_article.shape

In [ ]:
df_train_article_arr = df_train_article.values
df_train_article_arr.shape

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Merge, Activation, Dropout, Reshape

In [ ]:
# create user model
userModel1 = Sequential()
userModel1.add(Dense(500, input_shape=(2394,)))
userModel1.add(Activation('tanh'))
userModel1.add(Dropout(0.4))
userModel1.add(Dense(50, input_shape=(500,)))
userModel1.add(Activation('tanh'))
userModel1.add(Dropout(0.4))
userModel1.add(Dense(30, input_shape=(50,)))
userModel1.add(Activation('tanh'))
userModel1.add(Dropout(0.4))
userModel1.add(Dense(10, input_shape=(30,)))
userModel1.add(Activation('tanh'))
userModel1.add(Reshape((10,1)))

In [ ]:
# create article model
articleModel = Sequential()
articleModel.add(Dense(50, input_shape=(187,)))
articleModel.add(Activation('tanh'))
articleModel.add(Dropout(0.4))
articleModel.add(Dense(30, input_shape=(50,)))
articleModel.add(Activation('tanh'))
articleModel.add(Dropout(0.4))
articleModel.add(Dense(10, input_shape=(30,)))
articleModel.add(Activation('tanh'))
articleModel.add(Reshape((10,1)))

In [ ]:
model1 = Sequential()
model1.add(Merge([userModel1,articleModel], mode='cos', dot_axes=1, output_shape=(2,)))
model1.add(Reshape((1,)))

In [ ]:
# Compile model
model1.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy'])

In [ ]:
len(df_test_article_arr1)

In [ ]:
model1.fit([df_train_user_arr ,df_train_article_arr] ,y_train, nb_epoch=1, batch_size=32)

In [ ]:
model1.fit([df_train_user_arr ,df_train_article_arr] ,y_train, nb_epoch=1, batch_size=32)

In [ ]:
# Compile model 1
#model1.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy','precision','recall'])
#model1.fit([df_train_user_arr ,df_train_article_arr] ,y_train, nb_epoch=1, batch_size=32)

# Compile model 2
#model2.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy','precision','recall'])
#model2.fit([df_train_app_arr ,df_train_article_arr] ,y_train, nb_epoch=1, batch_size=32)

In [ ]:
#y_pred = model.predict([df_test_user,df_test_article_arr1, df_test_app], batch_size=32, verbose=0)
#y_pred_1 = np.hstack(y_pred)
#np.argsort(y_pred_1)[:100]

### Calculate precision and recall

In [ ]:
df_test_user_arr1 = df_test_user.values
df_test_user_arr1.shape

In [ ]:
#sophia remark get the mapping of user article
#df_ua_map = df_trData[['user_id', 'article_contentid']]

#Sophia remark: get test user list
#df_test_user0 = df_user[df_user['user_id'].isin(final_test_user_id)]
#df_test_user = df_user[df_user['user_id'].isin(final_test_user_id)]
#df_test_user = df_test_user.drop(['user_id'], axis=1)
#df_test_user = df_test_user.fillna(0)
#df_test_user_arr1 = df_test_user[user_fea_col].values
#df_test_user_arr1.shape

#Sophia remark: get test app list
#df_test_app0 = df_app2[df_app2['user_id'].isin(test_user_id)]
#df_test_app = df_user[df_user['user_id'].isin(test_user_id)].merge(df_test_app0, how = 'left', on = 'user_id')[app_category]
#df_test_app = df_test_app.fillna(0)
#df_test_app_arr1 = df_test_app.values
#df_test_app_arr1.shape

#sophia remark: get article list
#df_test_article0 = df_tst_article.drop_duplicates(subset=['article_contentid'], keep='first')#[article_feature]
#df_test_article0.head()
col = df_train_article.columns
df_test_article_arr1 = df_test_article[col].values

In [ ]:
df_test_article[col].shape

In [ ]:
print df_test_article[col].shape
df_test_article[col].head()

In [ ]:
#df_train_article.head()

In [ ]:
df_test_article_arr1.shape

In [ ]:
#df_test_app_arr1.shape

In [ ]:
df_test_user0 = df_tst_user

In [ ]:
#df_test_article_arr1[0:5]

In [ ]:
l = len(df_test_article_arr1)
uid = df_test_user0['user_id'].tolist()[0]
user = np.array([df_test_user_arr1[0]]*l)
user
app = np.array([df_test_app_arr1[0]]*l)
#apply user and article array to do predict
up = pd.DataFrame(model.predict([user,df_test_article_arr1,app], batch_size=32))
df_test_article1=df_test_article0.reset_index(drop = True)\
                            .merge(up, how = 'left', left_index=True, right_index=True)\
                            .rename(columns={0:'u0_score'})[['article_contentid', 'u0_score']]
df_test_article1 = df_test_article1.sort_values(by='u0_score', ascending=False)
df_test_article1['y_pred'] = 0
df_test_article1[:5]['y_pred']=1
top5_art = df_test_article1[:5]['article_contentid'].values


top5_art

In [ ]:
l = len(df_test_article_arr1)
dic = {}
for idx, user in enumerate(df_test_user_arr1):
    #for each user-article pair: get the user id and user info
    uid = df_test_user0['user_id'].tolist()[idx]
    user = np.array([df_test_user_arr1[idx]]*l)
    app = np.array([df_test_app_arr1[idx]]*l)
    #apply user and article array to do predict
    up = pd.DataFrame(model.predict([user,df_test_article_arr1, app], batch_size=32))
    df_test_article1=df_test_article0.reset_index(drop = True)\
                            .merge(up, how = 'left', left_index=True, right_index=True)\
                            .rename(columns={0:'u0_score'})[['article_contentid', 'u0_score']]
    df_test_article1 = df_test_article1.drop_duplicates(['article_contentid'])
    df_test_article1 = df_test_article1.sort_values(by='u0_score', ascending=False)
    #df_test_article1['y_pred'] = 0
    dic[uid] = df_test_article1[:5]['article_contentid']
    print uid, ":", dic[uid].tolist()


In [ ]:
l = len(df_test_article_arr1)
precision = list()
recall = list()

for idx, user in enumerate(df_test_user_arr1):
    if idx > 6:
        break;
    #for each user-article pair: get the user id and user info
    uid = df_test_user0['user_id'].tolist()[idx]
    user = np.array([df_test_user_arr1[idx]]*l)
    app = np.array([df_test_app_arr1[idx]]*l)
    #apply user and article array to do predict
    up = pd.DataFrame(model.predict([user,df_test_article_arr1, app], batch_size=32))
    df_test_article1=df_test_article0.reset_index(drop = True)\
                            .merge(up, how = 'left', left_index=True, right_index=True)\
                            .rename(columns={0:'u0_score'})[['article_contentid', 'u0_score']]
    df_test_article1 = df_test_article1.sort_values(by='u0_score', ascending=False)
    df_test_article1['y_pred'] = 0
    df_test_article1[:5]['y_pred']=1
    df_match = df_ua_map[df_ua_map['user_id'] == uid][['article_contentid']]
    df_match['y_true'] = 1
    df_test_article1 = df_test_article1.merge(df_match, how = 'left', on = 'article_contentid')\
                                       .fillna(0)
    tp = df_test_article1[(df_test_article1['y_pred'] == 1) & (df_test_article1['y_true'] == 1)].shape[0]
    fp = df_test_article1[(df_test_article1['y_pred'] == 1) & (df_test_article1['y_true'] == 0)].shape[0]
    fn = df_test_article1[(df_test_article1['y_pred'] == 0) & (df_test_article1['y_true'] == 1)].shape[0]

    p = tp/(float(tp+fp))
    r = tp/(float(tp+fn))
    print (idx, uid, p,r)
    if p != 0:
        precision.append(p)
    if r != 0:
        recall.append(r)
    
print (precision, recall)

In [ ]:
df_test_article1.head()

In [ ]:
df_test_article1[df_test_article1['article_contentid'].isin(list(df_trData[df_trData['user_id'] == 'B3B51127D5FB5B39E4273BE65CACA28C']['article_contentid']))]

In [ ]:
df_ua_map[df_ua_map['user_id'] == '5BA72C11B1E94BA50F1E66F4B1362CE6']

In [ ]:
list(df_trData[df_trData['user_id'] == '5BA72C11B1E94BA50F1E66F4B1362CE6']['article_contentid'])

In [ ]:
t_u.head()

In [ ]:
t_u['index'] = t_u.index
t_u_group = t_u.groupby(by=[i for i in t_u.columns if i != 'index']).agg({'index':'nunique'}).reset_index(drop=True)
t_u_group = t_u_group.sort_values(by=['index'], ascending=False)
t_u_group.head()

# Deal with test data

In [ ]:
# get df_test_user

In [ ]:
# get df_test_app

In [ ]:
# get df_test_article

In [ ]:
'''
for idx, user in enumerate(df_test_user_arr2):
    if idx > 3:
        break;
     #for each user-article pair: get the user id and user info
    uid = df_test_user2['user_id'].tolist()[idx]
    user = np.array([df_test_user_arr2[idx]]*l)
    app = np.array([df_test_app_arr2[idx]]*l)
    
    #apply user and article array to do predict
    up = pd.DataFrame(model.predict([user,df_test_article_arr2, app], batch_size=32, verbose=0))
    df_test_article1=df_test_article0.reset_index(drop = True)\
                            .merge(up, how = 'left', left_index=True, right_index=True)\
                            .rename(columns={0:'u0_score'})[['article_contentid', 'u0_score']]
    df_test_article1 = df_test_article1.sort_values(by='u0_score', ascending=False)
    df_test_article1[:5]
'''